In [175]:
from flask import Flask,request
import mysql.connector
from flask_socketio import SocketIO,send,emit
from uuid import uuid4
import subprocess
import base64
from flask import send_file
import wave
import numpy as np
from sklearn.decomposition import FastICA,PCA, TruncatedSVD, KernelPCA
from sklearn.pipeline import Pipeline
import tensorflow as tf
from keras import Model, Sequential
from keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed

In [182]:
from scipy.io import wavfile as wav
from scipy.fftpack import fft
import librosa
from IPython.display import Audio
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os
import warnings
from sklearn.covariance import MinCovDet
warnings.filterwarnings("ignore")
f = {0: ["0.wav", "1.wav", "2.wav"], 1 : ["3.wav", "4.wav", "5.wav"], 2 :["7.wav","8.wav", "9.wav", "10.wav", "11.wav"],
    3: ["12.wav", "13.wav"], 5 : ["6.wav"]}
d = {0 : ["Rain Check", "https://www.youtube.com/watch?v=cYBCrCGw2b0"] , 
        1 : ["Kreepa Oh No", "https://www.youtube.com/watch?v=cYBCrCGw2b0"] , 
        3 : ["The Astronaut" , "https://www.youtube.com/watch?v=c6ASQOwKkhk"] , 2 : 
         ["Blinding Lights", "https://www.youtube.com/watch?v=4NRXx6U8ABQ"],
         5: ["Random Short", "np"]}
x = []
labels=[]
for i in os.listdir("tests"):
    if(i.endswith(".wav")):
        data, rate = librosa.load(f"tests/{i}")
        ms = librosa.feature.melspectrogram(data, rate)
        a = TruncatedSVD(7)
        sig = a.fit_transform(ms)
        x.append(sig.flatten())
        labels.append([j for j in f if i in f[j]][0])
labels = np.array(labels)
k = KMeans(init ='k-means++', n_clusters= len(d))
k.fit(x)
pred_classes = k.predict(x)
for cluster in range(len(d)):
    print('cluster: ', cluster)
    print(labels[np.where(pred_classes == cluster)])

cluster:  0
[2 2]
cluster:  1
[0 0 0]
cluster:  2
[2 3 2 2]
cluster:  3
[1]
cluster:  4
[3 1 1 5]


In [251]:
import scipy
librosa
f = {0: ["0.wav", "1.wav", "2.wav"], 1 : ["3.wav", "4.wav", "5.wav"], 2 :["9.wav","10.wav", "11.wav", "12.wav", "13.wav"],
    3: ["14.wav", "15.wav"], 5 : ["6.wav"], 6: ["7.wav", "8.wav"]}
d = {0 : ["Rain Check", "https://www.youtube.com/watch?v=cYBCrCGw2b0"] , 
        1 : ["Kreepa Oh No", "https://www.youtube.com/watch?v=cYBCrCGw2b0"] , 
        3 : ["The Astronaut" , "https://www.youtube.com/watch?v=c6ASQOwKkhk"] , 2 : 
        ["Blinding Lights", "https://www.youtube.com/watch?v=4NRXx6U8ABQ"],
        5: ["Random Short", "np"], 6: ["Without Me" , "https://www.youtube.com/watch?v=tqxRidAWER8&ab_channel=Eminem-Topic"]}
x = []
labels=[]
for i in os.listdir("tests"):
    if(i.endswith(".wav")):
        data, rate = librosa.load(f"tests/{i}")
        ms = librosa.feature.melspectrogram(data, rate)
        a = TruncatedSVD(7)
        sig = a.fit_transform(ms)
        x.append(sig.flatten())
        labels.append([j for j in f if i in f[j]][0])
labels = np.array(labels)
k = KMeans(init ='k-means++', n_clusters= len(d))
k.fit(x)
c,rate = librosa.load("tests/7.wav")

ms = librosa.feature.melspectrogram(c)

a = TruncatedSVD(7)
sig = a.fit_transform(ms)
x= np.array(x).astype('float64')
print(x.shape)
pred_classes = k.predict(x)
print(sig.flatten().reshape(1,-1).shape)
p = k.predict(sig.flatten().reshape(1,-1).astype('float64'))

scipy.stats.mode(labels[np.where(pred_classes == p)]), 1-k.score(sig.flatten().reshape(1,-1).astype('float64'))

(16, 896)
(1, 896)


(ModeResult(mode=array([6]), count=array([1])), -6.065597562441371e-06)

In [248]:
data, rate = librosa.load("tests/7.wav")
data2, rate2 = librosa.load("tests/8.wav")
ms1 = librosa.feature.melspectrogram(data, rate)
ms2 = librosa.feature.melspectrogram(data2, rate2)


print(ms1.shape, ms2[0:,0:486].shape)

S_filter = librosa.decompose.nn_filter(np.minimum(ms1,ms2[0:,0:486]),
                                       aggregate=np.median,
                                       
                                       width=int(librosa.time_to_frames(1, sr=rate)))
# S_filter = np.minimum(S_filter, ms2)
ms1 = librosa.util.softmask(ms1, np.abs(ms1 - ms2[0:,0:486])/4, power=3)
a = librosa.feature.inverse.mel_to_audio(ms1/2)

wav.write("sig1.wav", rate, a)

(128, 486) (128, 486)


In [125]:
print(n.shape)
inputs = Input(shape=())
encoded = LSTM(100)(inputs)
decoded = RepeatVector()(encoded)
decoded = LSTM(2, return_sequences=True)(decoded)
model = Model(inputs,decoded)


model.add(TimeDistributed(LSTM(100, activation='relu')))
model.add(RepeatVector(10))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
model.fit()

(1290240,)


ValueError: Input 0 of layer "lstm_1" is incompatible with the layer: expected ndim=3, found ndim=1. Full shape received: (None,)